In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

# cross_validate로 교차검증
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier # ensemble의 랜덤포레스트분류, 기본적으로 100개의 결정트리를 훈련함.
rf = RandomForestClassifier(n_jobs=-1,random_state=42) # 결정 트리의 앙상블이어서, 결정 트리의 max_depth, criterion 등등을 다 지원한다.
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1) # return_train_score은 검증 점수 뿐만 아니라 훈련 점수도 같이 반환한다.
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 0.99, 0.89 훈련 세트에 과대 적합되어 있음 (5개씩 쪼갠 배열의 평균)

rf.fit(train_input, train_target)
print(rf.feature_importances_) # 특성 중요도 출력 [0.23167441 0.50039841 0.26792718]
# -> 결정 트리보다 중요도가 골고루 되어있는데 랜덤으로 케이스들을 골랐기 때문이다. -> 과대적합을 줄이고 일반화 성능을 높인다!

rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42) # 복원추출로 뽑은 부트스트랩 샘플에 포함되지 않는 OOB(out of bag) 샘플로 평가를 할 수 있다.
rf.fit(train_input, train_target)
print(rf.oob_score_) # 각 결정 트리의 OOB 점수를 평균하여 출력한다.

# 엑스트라 트리 : 부트스트랩 샘플을 사용하지 않고 각 결정 트리를 만들 때 전체 훈련 세트를 사용한다. 노드를 분할할 때 무작위로 분할한다.
# 랜덤 포레스트가 특성을 기반으로 결정 트리를 무작위로 선택하는 것이였다면, 엑스트라 트리는 특성 또한 무작위로 선택하게 된다. 즉 무작위성을 더 추가한다는 뜻 -> 속도가 빠르다.
# 어렵게 말하면 후보 특성을 무작위로 분할하여 최적의 분할을 찾는 것이다.
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1) # 교차 검증해서 훈련, 검증 케이스의 점수 배열 반환
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 0.99 0.88

et.fit(train_input, train_target)
print(et.feature_importances_) # 특성 중요도 출력 [0.20,0.52,0.27]

# 그레이디언트 부스팅 : 깊이가 얕은 결정 트리를 사용하여 이진 트리의 오차를 보완(기본 깊이 3, 결정트리 100개) 과대적합에 강하다.
# 결정트리를 계속 추가하면서 가장 낮은곳을 찾아 이동한다. 경사 하강법을 이용해 트리를 앙상블에 추가한다. 분류 = 로지스틱 손실함수, 회귀 = 평균제곱오차함수 사용
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1) # 교차 검증
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 0.88 0.87 -> 과대적합에 강함

# 결정 트리의 개수를 늘려도 과대적합을 잘 억제함.
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42) # n_estimators = 500 : 결정 트리 개수 500개
# subsample = 0.8 -> 전체 데이터의 80%만 사용(기본은 100%), learning_rate = 0.2 학습률 20%(기본은 10%)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
gb.fit(train_input, train_target)
print(gb.feature_importances_) # 특성 중요도 출력 [0.15 0.68 0.16]

# 그레이디언트 부스팅은 순서대로 트리를 추가해서 훈련 속도가 느리고 n_jobs 매개변수가 없음. 성능을 개선한 것이 히스토그램 기반 그레이디언트 부스팅이다.
# 히스토그램 기반 그레이디언트 부스팅 : 정형 데이터를 다루는 머신러닝 중 가장 인기 있음, 입력 특성을 256개의 구간으로 나누어서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.
from sklearn.experimental import enable_hist_gradient_boosting # 이걸 먼저 해야함.
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 0.93 0.88

# permutation_importance() = 특성 중요도 계산하기 위해 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰해 어떤 특성이 중요한지를 계산한다.
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1) # n_repeats = 10 랜덤하게 섞을 횟수 : 10
print(result.importances_mean) # 특성 중요도 출력 [0.08, 0.23, 0.08] (특성중요도, 평균, 표준편차)

result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean) # 특성 중요도 출력 [0.05 0.2 0.04] (중요도, 평균, 표준편차)
hgb.score(test_input,test_target) # 0.87

# 히스토그램 기반 그레이디언트 부스팅 회귀버전은 HistGradientBoostingRegressor 클래스에 있음
# XGBOOST
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True) # 훈련 점수도 배열로 scores에 반환해줌.
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 0.95 0.87

#LightGBM
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809
[0.23167441 0.50039841 0.26792718]
0.8934000384837406
0.9974503966084433 0.8887848893166506
[0.20183568 0.52242907 0.27573525]
0.8881086892152563 0.8720430147331015
0.9464595437171814 0.8780082549788999
[0.15872278 0.68010884 0.16116839]


/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236
[0.08876275 0.23438522 0.08027708]
[0.05969231 0.20238462 0.049     ]


0.8723076923076923